# Preparando dados de número de votos e receita

In [13]:
import pandas as pd
from glob import glob

import xavy.utils as xu

## Funções

In [2]:
def agg_votos(df, cargos=['DEPUTADO FEDERAL', 'DEPUTADO ESTADUAL'], agg_by=['SQ_CANDIDATO'], turno=1):

    # Filter candidates:
    df = df.loc[df['DS_CARGO'].isin(cargos)]
    df = df.loc[df['NR_TURNO'] == turno]
    assert df['NM_TIPO_ELEICAO'].nunique() <= 1, 'Encontrado mais um tipo de eleição. Verificar.'

    # Count votes:
    total_votos = df.groupby(agg_by)['QT_VOTOS_NOMINAIS'].sum()

    return total_votos

In [3]:
def etl_votos(filename):
    # Load data:
    df = pd.read_csv(filename)
    
    # Aggregate votes:
    votos = agg_votos(df)

    return votos

## Receita

In [4]:
# Carrega dados:
receita_df = pd.read_csv('../dados/brutos/receitas_candidatos_2022_BRASIL.csv', sep=';', encoding='latin-1', low_memory=False)

In [16]:
# Transforma valores em float:
receita_df['VR_RECEITA'] = xu.parse_ptbr_series(receita_df['VR_RECEITA'])

In [20]:
# Agrega receita por candidato:
total_receita_df = receita_df.groupby('SQ_CANDIDATO')['VR_RECEITA'].sum().reset_index()

In [22]:
# Salva:
#total_receita_df.to_csv('../dados/processados/receita_por_candidato_2022.csv', index=False)

## Agregando votos por candidato

### 2018

In [ ]:
# Lista os votos:
votos_files = glob('../dados/limpos/votos_2018/votacao_candidato_munzona_2018_??.csv')

# Carrega votos:
votos_list = [etl_votos(f) for f in votos_files]
votos = pd.concat(votos_list)
votos.name = 'QT_VOTOS_NOMINAIS'

In [45]:
# Save:
#votos.reset_index().to_csv('../dados/processados/votos_por_candidato_2018.csv', index=False)

### 2022

In [4]:
df = pd.read_csv('../../violentometro/dados/brutos/tse/votacao_candidato_munzona_2022/votacao_candidato_munzona_2022_BRASIL.csv', sep=';', encoding='latin-1')

In [9]:
votos = agg_votos(df, cargos=['Deputado Estadual', 'Deputado Federal'])

In [11]:
# Save:
votos.reset_index().to_csv('../dados/processados/votos_por_candidato_2022.csv', index=False)